# Contenedores en Desarrollo

## Problemas

* Línas de comando muy largas `docker run .....`
* Si cada contenedor corre un servicio, tenemos que levantar tantos contenedores como componentes tenga nuestra aplicación, y luego dentenerlos... 😫
* Definición de redes manuales
* Gestión de nombres
* etc.

# [Docker Compose](https://docs.docker.com/compose/overview/)

<img src="images/docker-compose-logo.png" width="200px">

1. Docker compose resuelve el problema de ejecutar **mútiples contenedores**, utilizando un archivo de definición de configuración [YAML](http://yaml.org/) (puede usar JSON u otros).
2. Utiliza comandos más simples:
    ```
    docker-compose up
    
    ```
3. Casi todas las opiciones de docker run están disponibles desde docker-compose.


Veamos un ejemplo...

In [4]:
%%bash
mkdir -p compose_wordpress

In [1]:
%%writefile compose_wordpress/docker-compose.yml

version: '3.1'

services:

  wordpress:
    image: wordpress
    ports:
      - 8080:80
    environment:
      WORDPRESS_DB_PASSWORD: example

  mysql:
    image: mysql:5.7
    environment:
      MYSQL_ROOT_PASSWORD: example


Overwriting compose_wordpress/docker-compose.yml


# Versión actual

In [5]:
%%bash
docker-compose --version

docker-compose version 1.14.0, build c7bdf9e


# Como se corre?

In [7]:
!cd compose_wordpress && docker-compose up

composewordpress_wordpress_1 is up-to-date
composewordpress_mysql_1 is up-to-date
Attaching to composewordpress_wordpress_1, composewordpress_mysql_1
wordpress_1  | WordPress not found in /var/www/html - copying now...
wordpress_1  | Complete! WordPress has been successfully copied to /var/www/html
wordpress_1  | 
wordpress_1  | MySQL Connection Error: (2002) Connection refused
wordpress_1  | 
wordpress_1  | Warning: mysqli::mysqli(): (HY000/2002): Connection refused in - on line 22
wordpress_1  | 
wordpress_1  | MySQL Connection Error: (2002) Connection refused
wordpress_1  | 
wordpress_1  | Warning: mysqli::mysqli(): (HY000/2002): Connection refused in - on line 22
wordpress_1  | 
wordpress_1  | Warning: mysqli::mysqli(): (HY000/2002): Connection refused in - on line 22
wordpress_1  | 
mysql_1      | Initializing database
wordpress_1  | MySQL Connection Error: (2002) Connection refused
mysql_1      | 2017-09-20T14:09:27.090269Z 0 [Warning] TIMESTAMP with implicit DEFAULT value is dep

wordpress_1  | 172.27.0.1 - - [20/Sep/2017:21:10:42 +0000] "GET / HTTP/1.1" 302 379 "http://localhost:8888/notebooks/Documents/Universidad/EIP2017/eip-virt-docker/03%20-%20Contenedores%20en%20Desarrollo.ipynb" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36"
wordpress_1  | 172.27.0.1 - - [20/Sep/2017:21:10:42 +0000] "GET /wp-admin/install.php HTTP/1.1" 200 4226 "http://localhost:8888/notebooks/Documents/Universidad/EIP2017/eip-virt-docker/03%20-%20Contenedores%20en%20Desarrollo.ipynb" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36"
wordpress_1  | 172.27.0.1 - - [20/Sep/2017:21:10:44 +0000] "GET /wp-includes/css/buttons.min.css?ver=4.8.1 HTTP/1.1" 200 1887 "http://localhost:8080/wp-admin/install.php" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36"
wordpress_1  | 1

# Accediendo a http://localhost:8080
![](images/wp.png)

# Qué está sucediendo?
<img src="images/dibujo.svg">

# ¿Que sucedió?
* Se crearon las imágenes:

    `docker images`
* Se crearon los contenedores:

    `docker ps`
* Se creó una red
* Se expusieron los puertos


In [12]:
!docker network ls

NETWORK ID          NAME                             DRIVER              SCOPE
0d007a2ab699        alpinenovnc_x11                  bridge              local
a4a43114cbc4        bridge                           bridge              local
3eab10ed6ac1        composeflask_default             bridge              local
8ada97acf5da        composewordpress_default         bridge              local
57d54b705ec8        coreemu_default                  bridge              local
77b9250aaac3        djangodocker_default             bridge              local
ff02eb41e98c        fixturerobots_default            bridge              local
3cf8a5d2ac28        flaskcelerydockerscale_default   bridge              local
2ad5dd1aa9c1        host                             host                local
aab12fe2f8d8        mqttrestcomp_default             bridge              local
ae72e1a98f95        mqttvsrestpapaertest_default     bridge              local
7702a7a28bf4        noderedpython_default           

# Un ejmplo un [poco más completo](https://docs.docker.com/compose/wordpress/#define-the-project)

```yaml
version: '3'

services:
   db:
     image: mysql:5.7
     volumes:
       - db_data:/var/lib/mysql
     restart: always
     environment:
       MYSQL_ROOT_PASSWORD: somewordpress
       MYSQL_DATABASE: wordpress
       MYSQL_USER: wordpress
       MYSQL_PASSWORD: wordpress

   wordpress:
     depends_on:
       - db
     image: wordpress:latest
     ports:
       - "8000:80"
     restart: always
     environment:
       WORDPRESS_DB_HOST: db:3306
       WORDPRESS_DB_USER: wordpress
       WORDPRESS_DB_PASSWORD: wordpress
volumes:
    db_data:
```

# Desarrollando con Docker
* Muchos frameworks tienen servidores que *recompilan* nuestro códigos con los cambios que hacemos.
* Para esto es útil utilizar un VOLUME para comparir el código fuente.
* Este enfoque requiere tener dos archivos docker-compose.yml:
    * uno para desarrollo con el VOLUME
    * uno para producción con un COPY

## Ejemplo

In [14]:
%%bash
mkdir -p compose_flask
mkdir -p compose_flask/code

In [15]:
%%writefile compose_flask/code/app.py

from flask import Flask
app = Flask(__name__)
@app.route("/")
def index():
    return "Desde compose!!!"

app.run(
    debug=True, 
    port=3000, 
    # Si no ponemos nada, solo escucha en 127.0.0.1
    host='0'
)

Overwriting compose_flask/code/app.py


In [56]:
%%writefile compose_flask/Dockerfile

FROM python
RUN pip install Flask
COPY ./code /code
#VOLUME /code
CMD ["python", "/code/app.py"]

Overwriting compose_flask/Dockerfile


In [11]:
%%writefile compose_flask/docker-compose.yml
version: '3'
    
services:
    web:
        build:
            context: .
        ports:
            - 3000:3000

Overwriting compose_flask/docker-compose.yml


In [13]:
!cd compose_flask && docker-compose down && docker-compose up

Removing composeflask_web_1 ... 
Removing network composeflask_default
Creating network "composeflask_default" with the default driver
Creating composeflask_web_1 ... 
Creating composeflask_web_1
Attaching to composeflask_web_12mdone
web_1  | python: can't open file '/code/app.py': [Errno 2] No such file or directory
composeflask_web_1 exited with code 2


### En produccción sería...

In [14]:
%%writefile compose_flask/Dockerfile

FROM python
RUN pip install Flask
COPY ./code /code  #VOLUME /code
CMD ["python", "/code/app.py"]

Overwriting compose_flask/Dockerfile


In [5]:
!cd compose_flask/ && docker-compose down

Stopping composeflask_web_1 ... 
Removing composeflask_web_1 ... mdone
Removing network composeflask_default


## Construyendo si es necesario y mandando al *fondo*
```
docker-compose up --build -d 
```

### Cómo vemos la salida?
```
docker-compose logs
```

### Cómo creamos una terminal
```
docker-compose exec web
```
(notar que no hace falta el -ti)

### Como bajamos la aplicación?
```
docker-compose down
```

## ¿Qué sucede si editamos el código?

Acá lanzamos un editor de texto o IDE de nuestra preferencia, en mi caso utilizaré [VS Code](https://code.visualstudio.com/#alt-downloads)
![](images/vscode.png)


In [67]:
!code compose_flask/

# Ejercicios